In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab1/ai_scholar/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
import csv
import numpy as np
import sys
from pathlib import Path
import pickle

In [ ]:
scholar_all = np.load(GOOGLE_DRIVE_PATH+'general/gs_scholars_all_upd.npy', allow_pickle=True)

In [ ]:
del scholar_all

In [ ]:
scholar_all[0]

# table tab:scholar_analysis

## academic age & lifespan analysis

In [ ]:
total_url_dict, female_url_dict = {}, {}
bug_check = []
bug_paper_check = []
for scholar in scholar_all:
  papers = scholar['papers']
  gender = df.loc[df['url'] == scholar['url'], 'gender'].item()
  year_set = set()
  for paper in papers:
    if paper[5] == '':
      continue
    if int(paper[5]) <= 1950:
      bug_paper_check.append((scholar['url'], int(paper[5])))
      continue
    if int(paper[5]) > 2022:
      continue
    year_set.add(int(paper[5]))
  if len(year_set) == 0:
    continue
    # year_first, year_last = 2023, 2022
    # bug_check.append(scholar['url'])
  year_first, year_last = min(year_set), max(year_set)

  total_url_dict[scholar['url']] = (year_first, year_last)
  if gender == 'F':
    female_url_dict[scholar['url']] = (year_first, year_last)

In [ ]:
bug_check

In [ ]:
bug_paper_check

In [ ]:
total_aca_age = pd.Series([2022-year[0]+1 for year in total_url_dict.values()])
total_aca_span = pd.Series([year[1] - year[0]+1 for year in total_url_dict.values()])
female_aca_age = pd.Series([2022-year[0]+1 for year in female_url_dict.values()])
female_aca_span = pd.Series([year[1] - year[0]+1 for year in female_url_dict.values()])

In [ ]:
total_aca_age.describe()

In [ ]:
female_aca_age.describe()

In [ ]:
female_aca_span.describe()

In [ ]:
total_aca_span.describe()

## top coauthors analysis

In [ ]:
!pip install efficiency

In [ ]:
class Name2Gender:
    def __init__(self):
        self.full_name2gender = self._load_full_name_gender()
        self.first_name2gender = self._load_first_name_gender()

    def _load_first_name_gender(self):
        # 100711 lines
        from collections import defaultdict
        df = pd.read_csv(GOOGLE_DRIVE_PATH+"data/gender_preprocess/firstname_gender.csv")
        first_name_n_gender = df.to_dict(orient='records')
        first_name2gender = {dic['name']: dic['gender'] for dic in first_name_n_gender}
        return first_name2gender

    def _load_full_name_gender(self):
        def _file2full_name(file):
            from efficiency.log import fread
            full_names = fread(GOOGLE_DRIVE_PATH+'data/gender_preprocess/'+file, delete_empty=True, if_strip=True)
            # full_names = U.read_file(file)
            full_names = [' '.join(i.split(', ', 1)[::-1]).lower() for i in full_names]
            return full_names

        gender2file = {'M': 'acl-male.txt',
                       'F': 'acl-female.txt',
                       }
        full_name2gender = {}
        for gender, file in gender2file.items():
            full_names = _file2full_name(file)
            full_name2gender.update({i: gender for i in full_names})
        return full_name2gender

    def lookup_gender(self, full_name):
        gender = self.full_name2gender.get(full_name.lower(), '-')
        if gender == '-':
            first_name = full_name.rsplit(' ', 1)[0].lower()
            gender = self.first_name2gender.get(first_name, '-')
        return gender

In [ ]:
name2gender = Name2Gender()

In [ ]:
name2gender.lookup_gender('Christian A. Lang')

In [ ]:
print(scholar_all[1]['co_authors'])

In [ ]:
percent_list = []
for scholar in scholar_all: # scholar_all is np.load() from 78k_scholar_new
  if name2gender.lookup_gender(scholar['name']) == 'F':
      co_authors = scholar['co_authors']
      if co_authors:
          male_count, female_count = 0, 0
          for each in co_authors:
              # coauthor_names.append(each[1])
              if name2gender.lookup_gender(each[1]) == 'M':
                  male_count += 1
              elif name2gender.lookup_gender(each[1]) == 'F':
                  female_count += 1         
          if male_count + female_count != 0:
              female_percentage = female_count / (male_count + female_count)
          elif male_count == 0 and female_count != 0:
              female_percentage = 1
          elif female_count == 0 and male_count != 0:
              female_percentage = 0
          else:
              continue
          percent_list.append(female_percentage)
percent_series = pd.Series(percent_list)
percent_series.describe()

# scholar_centric analysis

In [ ]:
# %%capture
import pandas as pd
df = pd.DataFrame.from_records(scholar_all)
df.head(1)

In [ ]:
df.head(5)

In [ ]:
# For each of the 78K scholars, “#co-authors/scholar_profile” report “avg+/- std”
for idx, row in df.iterrows():
  co_authors = row['co_authors']
  if co_authors is not None:
    df.at[idx, 'num_coauthors'] = len(co_authors)
  else:
    df.at[idx, 'num_coauthors'] = None

In [ ]:
df['num_coauthors']

In [ ]:
df['num_coauthors'].describe()

In [ ]:
df['num_coauthors'].isna().sum()

In [ ]:
# On average, report %co-authors can be found in gs_scholars.jsonl (⅕ or 1/10 missing)
print(df['num_coauthors'].isna().sum() / len(df['num_coauthors']))

In [ ]:
# Make sure that #co-authors do not differ #topics, otherwise we can normalized_topic_diversity = num_topics_among_all_coauthors/num_coauthors
for idx, row in df.iterrows():
  co_domain = row['domain_labels']
  if co_domain is not None:
    df.at[idx, 'num_domain'] = len(co_domain)
  else:
    df.at[idx, 'num_domain'] = None

In [ ]:
df['num_domain'].describe()

In [ ]:
# get normalized_topic_diversity
for idx, row in df.iterrows():
  co_author_list = row['co_authors']
  if co_author_list is None:
    df.at[idx, 'norm_topic_diversity'] = None
  else:
    sid_list = []
    for co_author in co_author_list:
      if 'user' in co_author[0]:
        sid_list.append(co_author[0].split('user=', 1)[1][:12])
    fields_set = set()
    for sid in sid_list:
      url = 'https://scholar.google.com/citations?hl=en&user='+sid
      if url in df['url'].values:
        domain = df.loc[df['url'] == url, 'domain_labels'].item()
        fields_set.update(set(domain))
    df.at[idx, 'norm_topic_diversity'] = len(fields_set)
# TODO: rename norm_topic_diversity to topic_diversity, compute norm_topic_diversity

In [ ]:
df['norm_topic_diversity']

In [ ]:
del df

In [ ]:
np.save(GOOGLE_DRIVE_PATH+'general/gs_scholars_all_upd.npy', df.to_dict('records'), allow_pickle=True)

## Another way of getting a full list of coauthors

In [ ]:
scholar_all[0]['papers'][1]

In [ ]:
from collections import defaultdict
paper_title2scholars_list = defaultdict(list)
for scholar in scholar_all:
  gs_sid = scholar['gs_sid']
  papers = scholar['papers']
  for paper in papers:
    title = paper[1]
    paper_title2scholars_list[title].append(gs_sid)

In [ ]:
with open(GOOGLE_DRIVE_PATH+'general/paper_title2scholars_list.pkl', 'wb') as file:
  pickle.dump(paper_title2scholars_list, file)

# new_cluster_data.jsonl (outdated)

*   列表项
*   列表项



In [ ]:
print(len(scholar_all))

In [ ]:
# if need sort:
def sort_by_cite(cluster_url):
  cluster_url.sort(key = lambda x: int(x['citation_table'][0]))
  return cluster_url[::-1]
sorted_scholars = sort_by_cite(scholar_all)

In [ ]:
# else:
sorted_scholars = scholar_all

In [ ]:
def get_field_by_index(index):
  return sorted_scholars[index]['domain_labels']

In [ ]:
def get_gssid_by_index(index):
  return sorted_scholars[index]['gs_sid']

In [ ]:
df_sorted_scholars = pd.DataFrame.from_records(sorted_scholars)
df_sorted_scholars

In [ ]:
df_all = pd.DataFrame.from_records(scholar_all)
df_all

In [ ]:
my_string="hello python worlda a , i'm a beginner "
print(my_string.split("world",1)[1])

In [ ]:
def get_field_by_gssid(sid):
  field_series = df_sorted_scholars.loc[df_sorted_scholars['gs_sid'] == sid, 'domain_labels']
  if field_series.empty:
    return []
  else:
    return field_series.item()

In [ ]:
# return a list of gs_sids of co_authors given sid
def get_coauthor_by_gssid(sid):
  co_author_list = df_all.loc[df_all['url'] == 'https://scholar.google.com/citations?hl=en&user='+sid, 'co_authors'].item()
  if co_author_list is None:
    return []
  sid_list = []
  for co_author in co_author_list:
    if 'user' in co_author[0]:
      sid_list.append(co_author[0].split('user=', 1)[1][:12])
  return sid_list

In [ ]:
get_coauthor_by_gssid(get_gssid_by_index(3))

In [ ]:
# scholar-centric analysis
def get_cofield_by_index(index):
  gssid = get_gssid_by_index(index)
  # while len(get_coauthor_by_gssid(gssid)) == 0:
  #   index += 1
  #   gssid = get_gssid_by_index(index)
  sid_list = get_coauthor_by_gssid(gssid)
  # add the author itself
  sid_list.append(gssid)
  # return all fields
  fields_set = set()
  # print(len(sid_list))
  for sid in sid_list:
    fields_set.update(set(get_field_by_gssid(sid)))
    # print(get_field_by_gssid(sid))
  return fields_set

In [ ]:
get_cofield_by_index(3)

In [ ]:
intpoint_array = np.arange(0, len(scholars), 1000)
avg_num_field = []
for intpoint in intpoint_array:
  num_field = []
  for point in range(intpoint, intpoint+100):
    num_field.append(len(get_cofield_by_index(point)))
  avg_num_field.append(np.mean(num_field, dtype=int))
avg_num_field

## plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
fig.canvas.draw()
ax.plot(np.arange(len(avg_num_field)), avg_num_field)
# ax.set_xticks([0, 20000, 40000, 68128])
ax.set_xticklabels([0, 0, 10000, 20000, 30000, 40000, 50000, 60000, 70000])
ax.set_xlabel('scholar rank sorted by total citations')
ax.set_ylabel('average number of fields')
plt.show()

# cluster analysis

## increase and then decrease

In [ ]:
# clusters 1, 6, 7 are increase and then decrease
import pickle
model_labels = []
with open(GOOGLE_DRIVE_PATH+'model_label.pkl', 'rb') as f:
    model_labels = pickle.load(f)
print(model_labels)

In [ ]:
len(model_labels) # should use scholars (new_cluster_data.jsonl)

In [ ]:
cluster1_idx_list, cluster6_idx_list, cluster7_idx_list = [], [], []
for i in range(len(model_labels)):
  if model_labels[i] == 1:
    cluster1_idx_list.append(i)
  elif model_labels[i] == 6:
    cluster6_idx_list.append(i)
  elif model_labels[i] == 7:
    cluster7_idx_list.append(i)

In [ ]:
# df_scholars = pd.DataFrame.from_records(scholars)
# df_scholars

In [ ]:
from collections import defaultdict
def gen_word_freq_dict(idx_list):
  word_freq_dict = defaultdict(int)
  for idx in idx_list:
    for label in scholars[idx]['domain_labels']:
      word_freq_dict[label] += 1
  word_freq_dict = {k: v for k, v in sorted(word_freq_dict.items(), key=lambda item: item[1], reverse=True)}
  return word_freq_dict

In [ ]:
gen_word_freq_dict(cluster1_idx_list)

In [ ]:
gen_word_freq_dict(cluster6_idx_list)

In [ ]:
gen_word_freq_dict(cluster7_idx_list)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
def gen_wordcloud(freq_dict, cluster_num):
  freq_list = []
  for key, value in freq_dict.items():
    freq_list += value * [key]
  freq_str = ' '.join(freq_list)
  # print(freq_str)
  # generate wordcloud
  wordcloud = WordCloud(width=5000, height=4000, collocations=False, random_state=10).generate(freq_str)
  plt.figure(figsize=(10, 8))
  plt.imshow(wordcloud)
  plt.title(f'cluster {cluster_num}')
  plt.axis('off')
  plt.savefig(f'cluster_{cluster_num}.pdf')

In [ ]:
gen_wordcloud(gen_word_freq_dict(cluster7_idx_list), 7)

### whether they no longer publish any new paper

In [ ]:
tmp = df_all.loc[df_all['url'] == 'https://scholar.google.com/citations?hl=en&user=bSU7LYoAAAAJ']
tmp

In [ ]:
def get_lastyear_dict(idx_list):
  year_dict = defaultdict(int)
  bug_check = []
  bug_paper_check = []
  for scholar_idx in idx_list:
    sid = scholars[scholar_idx]['gs_sid']
    url = 'https://scholar.google.com/citations?hl=en&user='+sid
    scholar_series = df_all.loc[df_all['url'] == url]
    papers = scholar_series['papers'].item()
    year_set = set()
    for paper in papers:
      if paper[5] == '':
        continue
      if int(paper[5]) <= 1950:
        bug_paper_check.append((url, int(paper[5])))
        continue
      if int(paper[5]) > 2022:
        continue
      year_set.add(int(paper[5]))
    if len(year_set) == 0:
      continue
      # year_first, year_last = 2023, 2022
      # bug_check.append(scholar['url'])
    year_last = max(year_set)
    year_dict[year_last] += 1
  return year_dict

In [ ]:
get_lastyear_dict(cluster7_idx_list)

F% among all coauthors

In [ ]:
import json
with open(GOOGLE_DRIVE_PATH+"gs_scholar_new_features.jsonl") as f:
  scholar_new = [json.loads(line) for line in f if line.strip()]

In [ ]:
import pandas as pd
df_scholar_new = pd.DataFrame.from_records(scholar_new)

In [ ]:
import pandas as pd
df_scholar = pd.DataFrame.from_records(scholar_all)

In [ ]:
df_scholar.head(2)

%F among top coauthors

In [ ]:
cnt_dict = {
  'f_in_f': [],
  'f_in_all': []
}
for cnt, scholar in enumerate(scholar_all):
  co_authors_all = scholar['co_authors']
  # print(co_authors_all)
  gender = scholar['gender']
  if co_authors_all is not None:
    tmp_list = []
    for co_authors in co_authors_all:
      tmp_list.append(co_authors[0].split('user=', 1)[1][:12])
    f_cnt = 0
    all_cnt = 0
    for co_author in tmp_list:
      tmp = df_scholar.loc[df_scholar['gs_sid'] == co_author, 'gender']
      if not tmp.empty:
        tmp = tmp.item()
        if tmp == 'F': 
          f_cnt += 1
        all_cnt += 1
      #   co_gender_list.append(tmp.item())
    if all_cnt == 0:
      continue
    cnt_ratio = f_cnt / all_cnt
    if gender == 'F':
      cnt_dict['f_in_f'].append(cnt_ratio)
    cnt_dict['f_in_all'].append(cnt_ratio)
  # if cnt >= 15:
  #   break

In [ ]:
import pickle
with open(GOOGLE_DRIVE_PATH+'scholar/f_coauthors_in_top.pkl', 'wb') as file:
  pickle.dump(cnt_dict, file)

In [ ]:
import pandas as pd
tmp = pd.DataFrame()
tmp['f_in_f'] = cnt_dict['f_in_f']
tmp2 = pd.DataFrame()
tmp2['f_in_all'] = cnt_dict['f_in_all']

In [ ]:
tmp.describe()

In [ ]:
tmp2.describe()

F% among all coauthors

In [ ]:
cnt_dict = {
  'f_in_f': [],
  'f_in_all': []
}
for cnt, scholar in enumerate(scholar_all):
  co_authors_all = scholar['co_authors_all']
  gender = scholar['gender']
  f_cnt = 0
  m_cnt = 0
  for co_author in co_authors_all:
    tmp = df_scholar.loc[df_scholar['gs_sid'] == co_author, 'gender']
    if not tmp.empty:
      tmp = tmp.item()
      if tmp == 'F': 
        f_cnt += 1
      elif tmp == 'M':
        m_cnt += 1
    #   co_gender_list.append(tmp.item())
  if f_cnt + m_cnt == 0:
    continue
  cnt_ratio = f_cnt / (f_cnt + m_cnt)
  if gender == 'F':
    cnt_dict['f_in_f'].append(cnt_ratio)
  cnt_dict['f_in_all'].append(cnt_ratio)
  # if cnt > 10:
  #   break

In [ ]:
import pickle
with open(GOOGLE_DRIVE_PATH+'scholar/f_coauthors_in_all.pkl', 'wb') as file:
  pickle.dump(cnt_dict, file)

In [ ]:
import pickle
with open(GOOGLE_DRIVE_PATH+'scholar/f_coauthors_in_all.pkl', 'rb') as file:
  cnt_dict = pickle.load(file)

domain analysis

In [ ]:
nlp = [('natural_language_processing', 6963),
 ('computational_linguistics', 1200),
 ('nlp', 818),
 ('text_analytics', 59),
 ('text_analysis', 33),
 ('natural_language', 24),
 ('language_technology', 59),
 ('natural_language_understanding', 116),
 ('language_modeling', 39),
 ('document_analysis', 46)]

In [ ]:

total = 0
female = 0
male = 0
for index, row in df_scholar_new.iterrows():
  for nlp_item in nlp:
    if nlp_item[0] in row["domain_labels"]:
      total += 1
      if row["gender"] == 'M':
        male += 1 / len(row['domain_labels'])
      if row["gender"] == 'F':
        female += 1 / len(row['domain_labels'])
print(male)
print(female)
print(male/total)
print(female/total)

\# scholars vs. starting year to publish

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
df

In [ ]:
from collections import defaultdict
from typing import List
year_dict = defaultdict(int)
year_female_dict = defaultdict(int)
year_male_dict = defaultdict(int)
year_tot_dict = defaultdict(int)
for idx, row in df.iterrows():
  year_list = row['cites']['years']
  year_list = [int(year) for year in year_list]
  year_list.sort()
  # print(year_list)
  # break
  # print(year_list)
  tmp_year = 0
  for year in year_list:
    if year > 1950:
      tmp_year = year
      break
  if tmp_year == 0:
    print(row)
    continue
  year_dict[tmp_year] += 1
  if row['gender'] == 'F':
    year_female_dict[tmp_year] += 1
  elif row['gender'] == 'M':
    year_male_dict[tmp_year] += 1
  year_tot_dict[tmp_year] += 1

In [ ]:
year_tot_ratio_dict = defaultdict(int)
for year, tot_cnt in year_tot_dict.items():
  year_tot_ratio_dict[year] = year_female_dict[year] / tot_cnt

In [ ]:
year_bar10_tot_dict = defaultdict(int)
for year, cnt in year_tot_dict.items():
  if year < 1990:
    i = 1990
    if year < i:
      year_bar10_tot_dict[i-10] += cnt
  else:
    i = (year // 10) * 10 + 10
    if year < i and year + 10 >= i:
      year_bar10_tot_dict[i-10] += cnt

In [ ]:
year_bar10_tot_dict

In [ ]:
year_bar10_female_dict = defaultdict(int)
for year, cnt in year_female_dict.items():
  if year < 1990:
    i = 1990
    if year < i:
      year_bar10_female_dict[i-10] += cnt
  else:
    i = (year // 10) * 10 + 10
    if year < i and year + 10 >= i:
      year_bar10_female_dict[i-10] += cnt

In [ ]:
year_bar10_female_dict

In [ ]:
year_bar10_male_dict = defaultdict(int)
for year, cnt in year_male_dict.items():
  if year < 1990:
    i = 1990
    if year < i:
      year_bar10_male_dict[i-10] += cnt
  else:
    i = (year // 10) * 10 + 10
    if year < i and year + 10 >= i:
      year_bar10_male_dict[i-10] += cnt

In [ ]:
year_bar10_male_dict

In [ ]:
tmp_1 = np.array([1146, 5436, 24098, 46965, 421])
tmp_2 = np.array([72, 470, 2201, 4248, 45])
tmp_3 = np.array([654, 2901, 10692, 17680, 147])
tmp_4 = tmp_2+tmp_3
year_female = [sum(tmp_2[:idx+1])/sum(tmp_4[:idx+1]) for idx in range(len(tmp_1))]

In [ ]:
year_female

In [ ]:
year_tot_ratio_dict

In [ ]:
year_female_dict

In [ ]:
year_tot_dict

In [ ]:
import collections
od = collections.OrderedDict(sorted(year_tot_dict.items()))

In [ ]:
def get_female(year_female_dict):
  import collections
  od = collections.OrderedDict(sorted(year_female_dict.items()))
  x1, y1 = [], []
  for i in od.items():
    x1.append(i[0])
    y1.append(i[1])
  y1 = np.array(y1)
  y1_accumul = np.array([np.sum(y1[:i+1]) for i in range(len(y1))])
  return x1, y1_accumul

In [ ]:
tmp_x1, tmp_y1 = get_female(year_female_dict)

In [ ]:
tmp_x1[9], tmp_x1[19], tmp_x1[29], tmp_x1[39], tmp_y1[9], tmp_y1[19], tmp_y1[29], tmp_y1[39]

In [ ]:
x1, y1 = [], []
for i in od.items():
  x1.append(i[0])
  y1.append(i[1])

In [ ]:
y1 = np.array(y1)
y1_accumul = np.array([np.sum(y1[:i+1]) for i in range(len(y1))])

In [ ]:
y1, y1_accumul, len(y1), len(y1_accumul), np.array(x1)

In [ ]:
x1[1], x1[11], x1[21], x1[31], x1[41], y1_accumul[1], y1_accumul[11], y1_accumul[21], y1_accumul[31], y1_accumul[41]

In [ ]:
[year_female[0] * y1_accumul[1], year_female[1] * y1_accumul[11], year_female[2] * y1_accumul[21], year_female[3] * y1_accumul[31], year_female[4] * y1_accumul[41]]

In [ ]:
tmp_y = [2, 1393, 7685, 35295, 78051]

In [ ]:
year_female

In [ ]:
tmp_y, year_female

In [ ]:
np.array(tmp_y) * np.array(year_female)

In [ ]:
# year_female [1980, 1990, 2000, 2010, 2020]
# year_tot_dict
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots()
plot1, = ax.plot(x1, y1_accumul, color='black', label = 'Count of All Scholars')
ax.set_ylabel('# Scholars', fontsize=16)
ax.set_xlabel('Starting Year to Publish', fontsize=16)
# ax2 = ax.twinx()
plot2 = ax.bar([1980, 1990, 2000, 2010, 2020], [0.1983, 184.28, 1240.73, 6340.186, 14041.596], width=2, fill=True, color='#4393b0', label = 'Ratio of Female Scholars')
# ax2.set_ylim(0, 1)
ax.set_xlim(1975, 2025)
ax.set_ylim(0, 80000)
# ax2.set_ylabel('Female (%)', fontsize=16)
ax.tick_params(axis='both', labelsize=14)
# ax2.tick_params(axis='both', labelsize=14)
# plt.subplots_adjust(right=0.9)
# plt.subplots_adjust(bottom=0.15)
plt.tight_layout()
rects = ax.patches
labels = ["{:.2%}".format(item) for item in year_female]
cnt = 0
for rect, label in zip(rects, labels):
    height = rect.get_height()
    if cnt == 1:
      ax.text(
        rect.get_x() + rect.get_width() / 2-0.5, height+1200, label, ha="center", va="bottom", fontsize=12
      )
    elif cnt == 2:
      ax.text(
        rect.get_x() + rect.get_width() / 2+1, height-300, label, ha="center", va="bottom", fontsize=12
      )
    else:
      ax.text(
        rect.get_x() + rect.get_width() / 2, height+600, label, ha="center", va="bottom", fontsize=12
      )
    cnt += 1
# ax.text(1976, 10000, f'Count of female scholars', fontsize=14)
leg1 = ax.legend(fontsize=14)
leg2 = ax.legend(fontsize=14)

plt.show()
fig.savefig(GOOGLE_DRIVE_PATH+f'scholar/scholar_pub_trend.pdf')

probability vs number of all coauthors

In [ ]:
df.head(2)

In [ ]:
from collections import defaultdict
female_cnt_dict, male_cnt_dict, unclass_cnt_dict = defaultdict(int), defaultdict(int), defaultdict(int)
for idx, row in df.iterrows():
  num_coauthor = round(row['mean_num_coauthors_all'])
  if row['gender'] == '-':
    # print(row['num_coauthors_all_wwo_ai'])
    unclass_cnt_dict[num_coauthor] += 1
  elif row['gender'] == 'F':
    female_cnt_dict[num_coauthor] += 1
  else:
    male_cnt_dict[num_coauthor] += 1

In [ ]:
male_cnt_dict

In [ ]:
def get_bar_n_list(in_dict, bar=None, max_bar=None): # female_cnt100_dict, 100, 1000
  # print(tmp_dict)
  if bar is not None and max_bar is not None:
    tmp_dict = []
    for i in range(0, max_bar+bar, bar):
      tmp_dict.append([i, 0])
    for num_co, cnt in in_dict.items():
      if num_co > max_bar:
        i = max_bar
        tmp_dict[i//bar][1] += cnt
      else:
        i = (num_co // bar) * bar
        tmp_dict[i//bar][1] += cnt
        # print(tmp_dict, num_co, cnt)
        # break
  else:
    tmp_dict = []
    for i in range(0, max_bar+1, 1):
      tmp_dict.append([i, 0])
    for num_co, cnt in in_dict.items():
      if num_co > max_bar:
        i = int(max_bar)
        tmp_dict[i][1] += cnt
      else:
        i = int(num_co)
        tmp_dict[i][1] += cnt
        # print(tmp_dict, num_co, cnt)
        # break
  print(tmp_dict)
  return tmp_dict

In [ ]:
female_cnt100_list = get_bar_n_list(female_cnt_dict, max_bar=15)
male_cnt100_list = get_bar_n_list(male_cnt_dict, max_bar=15)
unclass_cnt100_list = get_bar_n_list(unclass_cnt_dict, max_bar=15)

In [ ]:
female_cnt100_list, male_cnt100_list, unclass_cnt100_list

In [ ]:
# only when max_bar = 15
unclass_cnt100_list.remove([0, 0])
male_cnt100_list.remove([0, 0])
female_cnt100_list.remove([0, 0])

In [ ]:
female_sum, male_sum, unclass_sum = sum(item[1] for item in female_cnt100_list), sum(item[1] for item in male_cnt100_list), sum(item[1] for item in unclass_cnt100_list)
female_sum, male_sum, unclass_sum

In [ ]:
nonfemale_cnt100_list = [[item1[0], item1[1]+item2[1]] for item1, item2 in zip(male_cnt100_list, unclass_cnt100_list)]
nonfemale_cnt100_list

In [ ]:
x = [i for i in range(1, 16, 1)]
female_prob = [round(item[1] / female_sum, 5) for item in female_cnt100_list]
nonfemale_prob = [round(item[1] / (male_sum+unclass_sum), 5) for item in nonfemale_cnt100_list]
male_prob = [round(item[1] / male_sum, 5) for item in male_cnt100_list]
female_prob, nonfemale_prob, male_prob

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig = plt.figure(figsize=(20, 10))
x_axis = np.arange(1, 13, 1)
x_axis_label = [str(x) for x in x_axis][:12]
# x_axis_label[-1] = '>320'
plt.rcParams.update({'font.size': 20})
# Multi bar Chart

plt.bar(x_axis -0.2, female_prob[:12], width=0.15, label = 'Female')
plt.bar(x_axis +0, nonfemale_prob[:12], width=0.15, label = 'Non-Female')
plt.bar(x_axis +0.2, male_prob[:12], width=0.15, label = 'Male')

# Xticks

plt.xticks(x_axis, x_axis_label)
plt.xlabel("number of author")
plt.ylabel("probability (/%)")
# Add legend

plt.legend()

# Display

plt.show()

probability vs number of all coauthors update (average num_papers per scholar)

In [ ]:
# count the number of all coauthors, including those are not in ai scholars
import numpy as np
for scholar in scholar_all:
  num_coauthors_list = []
  for paper in scholar['papers']:
    coauth_item_list = paper[2][0].split(', ')
    if coauth_item_list == ['...'] or coauth_item_list == ['']:
      continue
    if '...' in coauth_item_list:
      num_coauthors_list.append(len(coauth_item_list) - 1)
    else:
      num_coauthors_list.append(len(coauth_item_list))
  num_coauthors_list = np.array(num_coauthors_list)
  # print(num_coauthors_list[:100])
  mean_val = np.mean(num_coauthors_list)
  # print(mean_val)
  scholar['mean_num_coauthors_all'] = mean_val

In [ ]:
scholar_all[4]['mean_num_coauthors_all']

In [ ]:
import numpy as np
np.save(GOOGLE_DRIVE_PATH+'general/gs_scholars_all_upd_more.npy', scholar_all)

In [ ]:
# %%capture
import pandas as pd
df = pd.DataFrame.from_records(scholar_all)
df.head(1)

In [ ]:
df['mean_num_coauthors_all'].describe()

# coauthor diversity table

In [ ]:
pd.set_option('display.max_columns', 500)
df.head(2)

In [ ]:
import pandas as pd
df_causal = pd.read_csv(GOOGLE_DRIVE_PATH+"causal_upd_mv.csv")

In [ ]:
df_causal

In [ ]:
df_causal.describe()

In [ ]:
df_causal.loc[df_causal['gender']==1].describe()

In [ ]:
df_causal.loc[df_causal['co_female_ratio']==1]

In [ ]:
df_causal.loc[df_causal['co_num_coauthors_all']>=20].describe()

In [ ]:
df_causal.loc[(df_causal['co_num_coauthors_all']>=20) & (df_causal['gender'] == 1)].describe()

In [ ]:
df.loc[df['gender'] == 'F']

In [ ]:
df.loc[df['gender'] == 'M']